In [ ]:
!pip install contractions

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import nltk
import string
import fasttext
import contractions
import matplotlib as plt
import pyLDAvis.sklearn
from collections import Counter
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv('/kaggle/input/fake-news/train.csv')

In [ ]:
def pandas_profile(df):
    print(train_data.describe())
    print(train_data.info())
    from pandas_profiling import ProfileReport
    prof = ProfileReport(df)
    prof.to_file(output_file='report.html')

In [ ]:
pandas_profile(train_data)

In [ ]:
from IPython.display import HTML
report = './report.html'
HTML(filename=report)

In [ ]:
train_data.describe()

In [ ]:
train_data.info()

In [ ]:
def create_sequences(text):
    from tensorflow.keras.preprocessing.text import Tokenizer
    tok = Tokenizer()
    tok.fit_on_texts(text)
    sequence = tok.texts_to_sequences(text)
    return sequence
    

In [ ]:
def clean_data(sentence):
    if isinstance(sentence, str):
        clean_sentence = re.sub('[^A-Za-z0-9]+', ' ', sentence)
        return clean_sentence
    else:
        raise ValueError("error in sentence", sentence)

In [ ]:
train_data.dropna(subset = ["text"], inplace=True)
train_data['text'].apply(clean_data)


In [ ]:
train_data.head()

In [ ]:
def split_sentence(sentence):
    if isinstance(sentence, str):
        return sentence.split()

def join_sentence(sentence):
    return ' '.join(sentence)

In [ ]:
train_data['text_split'] = train_data['text'].apply(split_sentence)

In [ ]:
train_data['text_split']

In [ ]:
stop_words = set(stopwords.words('english'))
train_data['stopwords_removed'] = train_data['text_split'].apply(lambda x: [word for word in x if word not in stop_words])
train_data.head()

In [ ]:
words = train_data['stopwords_removed']
allwords = []
for wordlist in words:
    allwords += wordlist

In [ ]:
train_data['text_processed'] = train_data['stopwords_removed'].apply(join_sentence)

In [ ]:
train_data['text_processed']

In [ ]:
allwords

In [ ]:
mostcommon = FreqDist(allwords).most_common(100)
wordcloud = WordCloud(width=1600, height=800, background_color='white').generate(str(mostcommon))
fig = plt.figure(figsize=(30,10), facecolor='white')
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.title('Top 100 Most Common Words', fontsize=100)
plt.tight_layout(pad=0)
plt.show()

In [ ]:
mostcommon_small = FreqDist(allwords).most_common(25)
x, y = zip(*mostcommon_small)
plt.figure(figsize=(50,30))
plt.margins(0.02)
plt.bar(x, y)
plt.xlabel('Words', fontsize=50)
plt.ylabel('Frequency of Words', fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(rotation=60, fontsize=40)
plt.title('Frequency of 25 Most Common Words', fontsize=60)
plt.show()

In [ ]:
group_by = train_data.groupby('label')['text_processed'].apply(lambda x: Counter(' '.join(x).split()).most_common(25))
group_by_0 = group_by.iloc[0]
words0 = list(zip(*group_by_0))[0]
freq0 = list(zip(*group_by_0))[1]
plt.figure(figsize=(50,30))
plt.bar(words0, freq0)
plt.xlabel('Words', fontsize=50)
plt.ylabel('Frequency of Words', fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(rotation=60, fontsize=40)
plt.title('Frequency of 25 Most Common Words for not fake ', fontsize=60)
plt.show()

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.9, min_df=25, max_features=500)
tf = tf_vectorizer.fit_transform(train_data['text_processed'].values.astype('U'))
tf_feature_names = tf_vectorizer.get_feature_names()
# doc_term_matrix = pd.DataFrame(tf.toarray(), columns=list(tf_feature_names))
# doc_term_matrix

In [ ]:
lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', max_iter=500, random_state=0).fit(tf)
no_top_words = 10
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                          for i in topic.argsort()[:-no_top_words - 1:-1]]))
              
display_topics(lda_model, tf_feature_names, no_top_words)

In [ ]:
group_by = train_data.groupby('label')['text_processed'].apply(lambda x: Counter(' '.join(x).split()).most_common(25))
group_by_0 = group_by.iloc[1]
words0 = list(zip(*group_by_0))[0]
freq0 = list(zip(*group_by_0))[1]
plt.figure(figsize=(50,30))
plt.bar(words0, freq0)
plt.xlabel('Words', fontsize=50)
plt.ylabel('Frequency of Words', fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(rotation=60, fontsize=40)
plt.title('Frequency of 25 Most Common Words for fake', fontsize=60)
plt.show()

In [ ]:
def max_len(expressions):
    splitted_expressions = []
    splitted_expressions.extend(w.split(' ') for w in expressions)
    maxlen = len(max(splitted_expressions, key=len))
    return maxlen

In [ ]:
def prepare_X_y(df):
    df.dropna(subset = ["text"], inplace=True)
    df['text'].apply(clean_data)
    text = list(df['text'])
    sequences = create_sequences(text)
    maxlen = max_len(text)
    from keras.preprocessing import sequence
    X = sequence.pad_sequences(sequences, maxlen=maxlen, padding='post',
                                              truncating='post')
    y = df['label']
    y_train = np.asarray(y).astype('float32')
    return X , y_train
    
    

In [ ]:
X,y = prepare_X_y(train_data)

In [ ]:
len(X[2])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [ ]:
X_train.shape

In [ ]:

X_train1 = np.expand_dims(X_train, 2)

In [ ]:
X_train1.shape

In [ ]:
from keras import models
from keras import layers

def build_model():
    model = models.Sequential() 
    model.add(layers.LSTM(512, input_shape=(24234, 1), return_sequences=True)) 
    model.add(layers.LSTM(128, return_sequences=False))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc']) 
    history = model.fit(X_train1, y_train, validation_data=(X_test, y_test),epochs=10, batch_size=12, verbose=1)
    return history

In [ ]:
history = build_model()


In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history
loss_values = history_dict['loss'] 
acc = history_dict['acc'] 
val_loss_values = history_dict['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, loss_values, 'bo', label='Training loss') 
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss') 
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf() 
acc_values = history_dict['acc'] 
val_acc_values = history_dict['val_acc']
plt.plot(epochs, acc, 'bo', label='Training acc') 
plt.plot(epochs, val_acc_values, 'b', label='Validation acc') 
plt.title('Training and validation accuracy') 
plt.xlabel('Epochs')
plt.ylabel('Loss') 
plt.legend()
plt.show()